<a href="https://colab.research.google.com/github/donlap/stat424/blob/main/3_linear_program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.optimize import linprog

In [ ]:
a = np.ones(10)

a

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [ ]:
b = np.zeros(5)

b

array([0., 0., 0., 0., 0.])

In [ ]:
c = np.random.randn(5, 6)

c

array([[ 0.57535566,  0.87064906,  0.30696507,  0.96733902,  0.85490429,
        -0.63595636],
       [ 0.54538695, -0.20584613,  0.33761932, -0.05043866,  1.41676119,
         0.694284  ],
       [ 1.18213775,  0.59505434, -0.92723641,  0.85882576, -0.29897157,
         1.20175839],
       [-1.44950749, -0.67569349, -0.82316577, -0.61526767, -1.23622524,
        -0.75817492],
       [-0.97085423,  1.03308543,  0.89355903,  0.35027094, -1.86697532,
        -0.00764255]])

In [ ]:
d = np.random.randn(3, 4)
e = np.random.randn(3, 4)

f = np.vstack([d, e])
print(d)
print(e)
print(f)

[[-0.51578729 -1.15455683 -1.44374632  1.41243573]
 [ 0.39384886 -0.29509083 -1.20329075 -0.36298881]
 [ 0.83413796 -1.2976145  -1.24678848  0.22979005]]
[[ 0.87850272  0.69760691  0.82943072 -0.10963734]
 [ 1.25372853  0.24300609  1.08391715 -0.08366092]
 [-1.28017224  0.955438    1.51887127 -0.62402397]]
[[-0.51578729 -1.15455683 -1.44374632  1.41243573]
 [ 0.39384886 -0.29509083 -1.20329075 -0.36298881]
 [ 0.83413796 -1.2976145  -1.24678848  0.22979005]
 [ 0.87850272  0.69760691  0.82943072 -0.10963734]
 [ 1.25372853  0.24300609  1.08391715 -0.08366092]
 [-1.28017224  0.955438    1.51887127 -0.62402397]]


In [ ]:
g = np.hstack([d, e])

g

array([[-0.51578729, -1.15455683, -1.44374632,  1.41243573,  0.87850272,
         0.69760691,  0.82943072, -0.10963734],
       [ 0.39384886, -0.29509083, -1.20329075, -0.36298881,  1.25372853,
         0.24300609,  1.08391715, -0.08366092],
       [ 0.83413796, -1.2976145 , -1.24678848,  0.22979005, -1.28017224,
         0.955438  ,  1.51887127, -0.62402397]])

### Setup: Laptop company

Two types of laptop:
- Standard laptop: \$300 per unit sold
- Gaming laptop: \$500 per unit sold

Goal: Maximize profit

Constraints:

1. We have 1800 assembly hours. 3 hours for a standard laptop, 4 hours for a gaming laptop

2. We have 350 GPUs, which is only used to build a Gaming laptop

3. We have 1000 testing hours. Both types of laptop require 1 hour of testing.


In [ ]:
result = linprog(
    c=[-300, -500],                  # maximize 300*x1 + 500*x2 => minimize -300*x1 - 500*x2
    A_ub=np.array([[3, 4],           # constraint 1: 3*x1 + 4*x2 <= 1800
                   [0, 1],           # constraint 2: 0*x1 + 1*x2 <= 350
                   [1, 1]]),         # constraint 3: 1*x1 + 1*x2 <= 1000
    b_ub=[1800, 350, 1000],
    bounds=[(0, None), (0, None)]
)


In [ ]:
result

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -215000.0
              x: [ 1.333e+02  3.500e+02]
            nit: 0
          lower:  residual: [ 1.333e+02  3.500e+02]
                 marginals: [ 0.000e+00  0.000e+00]
          upper:  residual: [       inf        inf]
                 marginals: [ 0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00  5.167e+02]
                 marginals: [-1.000e+02 -1.000e+02 -0.000e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

Data: $(x_1, y_1), (x_2,y_2), \ldots, (x_n, y_n)$.

Model $\hat{y} = \beta_1 x_1 + \beta_2 x_2 + \ldots + \beta_m x_m = \beta^\top x$,
where $\beta = [\beta_1,\ldots,\beta_m]^\top$ and $x = [x_1, \ldots, x_m]^\top$

Optimize for $\beta$ by minimizing $\sum_{i=1}^n | y_i - \beta^\top x_i |$ with respect to $\beta$. **L1 Regression**.

Trick: use $|z| = \max\{z, -z\}$.

Let $u_i = | y_i - \beta^\top x_i |$. Then, $u_i \ge y_i - \beta^\top x_i$ and $u_i \ge -(y_i - \beta^\top x_i)$.

LP: $\min_{u,\beta} \sum_{i=1}^n u_i$ subject to:

1. $u_i \ge y_i - \beta^\top x_i$ or $u_i + \beta^\top x_i \ge y_i$ for $i=1\ldots,n$
2. $u_i \ge -(y_i - \beta^\top x_i)$ or $u_i - \beta^\top x_i \ge -y_i$ for $i=1\ldots,n$
3. $u_i \ge 0$ for $i=1\ldots,n$

Let $I$ be $n \times n$ identity matrix.
Let $X$ be the matrix with $x_i$ in the $i$-th row.
Let $u = [u_1, u_2, \ldots, u_n]^\top$

New constraints:
1. $-Iu - X \beta \le - y$
2. $-Iu + X \beta \le y$
3. $u \ge 0$



### Simulate data

In [ ]:
n, m = 50, 5
X = np.random.randn(n, m)
beta = np.array([2.0, 4.0, 6.0, 8.0, 10.0])
y = X @ beta + 0.2 * np.random.randn(n)

In [ ]:
result = linprog(
    c=np.hstack([np.ones(n), np.zeros(m)]),
    A_ub=np.vstack([np.hstack([-np.eye(n), -X]), np.hstack([-np.eye(n), X])]),
    b_ub=np.hstack([-y, y]),
    bounds=[(0, None)]*(m + n)
)

In [ ]:
result

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 7.738951999260608
              x: [ 3.275e-01  6.584e-02 ...  7.973e+00  9.994e+00]
            nit: 63
          lower:  residual: [ 3.275e-01  6.584e-02 ...  7.973e+00
                              9.994e+00]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 6.550e-01  1.317e-01 ...  0.000e+00
                              0.000e+00]
                 marginals: [-0.000e+00 -0.000e+00 ... -1.000e+00
                             -7.987e-01]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_

In [ ]:
beta

array([ 2.,  4.,  6.,  8., 10.])

In [ ]:
estimated_beta = result.x[-5:]

estimated_beta

array([1.95496117, 4.03626326, 6.00223466, 7.97272696, 9.99397049])